In [1]:
import matplotlib
#%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

/home/ubuntu/tacotron2/plotting_utils.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/ubuntu/anaconda3/envs/tts/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tts/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505

In [2]:
import os

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050
checkpoint_path = "checkpoint_15000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path, map_location = 'cpu')['state_dict'])  #added map_location = 'cpu'
_ = model.eval()          #it was originally model.cuda().eval().half()
waveglow_path = 'waveglow_256channels.pt'
waveglow = torch.load(waveglow_path, map_location = 'cpu')['model']   #added map_location = 'cpu'
waveglow.eval()   #originally waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
#denoiser = Denoiser(waveglow)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'glow.WaveGlow' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/envs/tts/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'glow.WN' has changed. you can retrieve the original source code by accessing the object's s

In [4]:
import soundfile as sf

In [5]:
import time

In [6]:
def convert(array):
    sf.write('sample.wav', array, 22050)
    os.system('ffmpeg -i {0} -filter:a "atempo=0.80" {1}'.format('sample.wav', 'sample0.wav'))
    #os.system('ffmpeg -i {0} -ar 8000 {1}'.format('sample0.wav', 'sample1.wav'))
    data, rate = sf.read('sample0.wav')
    os.remove('sample.wav')
    os.remove('sample0.wav')
    #os.remove('sample1.wav')
    return data

In [10]:
def speech(t):
    start = time.time()
    text = t
    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    print(sequence)
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).long()   #originally torch.from_numpy(sequence)).cuda().long()
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
    #audio_denoised = denoiser(audio, strength=0.01)[:, 0]
    data = convert(audio[0].data.cpu().numpy())
    #os.system('ffmpeg -i {0} -filter:a "atempo=0.85" {1}'.format('harvard_inference/audio/'+str(i)+'.wav', 'harvard_inference/audio_0.85/'+str(i)+'.wav'))
    aud = ipd.Audio(data, rate=22050)
    end = time.time()
    print(end-start)
    return aud

In [11]:
speech('I understand your frustration and disappointment. I am sorry that its happening and I would like to help prevent it in the future. What style of diapers did you buy? For instance, was it the snugglers, pull ups or baby dry.')

[[46 11 58 51 41 42 55 56 57 38 51 41 11 62 52 58 55 11 43 55 58 56 57 55
  38 57 46 52 51 11 38 51 41 11 41 46 56 38 53 53 52 46 51 57 50 42 51 57
   7 11 46 11 38 50 11 56 52 55 55 62 11 57 45 38 57 11 46 57 56 11 45 38
  53 53 42 51 46 51 44 11 38 51 41 11 46 11 60 52 58 49 41 11 49 46 48 42
  11 57 52 11 45 42 49 53 11 53 55 42 59 42 51 57 11 46 57 11 46 51 11 57
  45 42 11 43 58 57 58 55 42  7 11 60 45 38 57 11 56 57 62 49 42 11 52 43
  11 41 46 38 53 42 55 56 11 41 46 41 11 62 52 58 11 39 58 62 10 11 43 52
  55 11 46 51 56 57 38 51 40 42  6 11 60 38 56 11 46 57 11 57 45 42 11 56
  51 58 44 44 49 42 55 56  6 11 53 58 49 49 11 58 53 56 11 52 55 11 39 38
  39 62 11 41 55 62  7]]
21.24711561203003


NameError: name 'sequence' is not defined